# Imports

In [ ]:
!sudo apt-get install poppler-utils
!sudo apt install tesseract-ocr-eng

In [ ]:
!pip install pytesseract
!pip install pdf2image
!pip install transformers
!pip install bertviz
!pip install urllib3
!pip install sent2vec

In [ ]:
# Restart kernel after running above cells

In [ ]:
import numpy as np
import cv2
from PIL import Image as Img
from PIL import ImageDraw as Imgd
from PIL import ImageColor
from pdf2image import convert_from_path, convert_from_bytes
from IPython.display import display, Image
import pytesseract
import os
import shutil

In [ ]:
import torchvision.ops.boxes as bops
import torch
import pandas as pd
import time
import math

import nltk
nltk.download('punkt')
nltk.download('all')
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [ ]:
import torch
from bertviz import model_view

In [ ]:
from sent2vec.vectorizer import Vectorizer
vectorizer = Vectorizer()

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 5.2 MB/s eta 0:00:00


# Text Extraction from PDFs using Whitespace Algorithm

In [ ]:
#setting up tesseract for local systems
# tess_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# pytesseract.pytesseract.tesseract_cmd = tess_path
# print(pytesseract.get_languages(config='--tessdata-dir "C:/Program Files/Tesseract-OCR/tessdata"'))

In [ ]:
# function to segment rows
def rows(hsv,row,hsv_img,pg_no):
    st1=""
    st2=""
    f_st1=""
    f_st2=""
    whitespace = [[0,0,0]]
    p=0
    start=0
    for i in range(row):
        c=0
        for element in hsv[i]: #hsv[i] is a single row of pixels
            if element!=255: #checking for black pixel (hsv value for white = 255)
                c=1
                break
        if c==0:
            p+=1
        else:
            p+=1
            whitespace.append([p,start,i]) #storing position of whitespaces
            p=0
            c=0
            start=i
    whitespace.remove([0,0,0])
    whitespace_1 = [[0,0,0]]
    for ele,spos,epos in whitespace:
        if ele!=1:
            whitespace_1.append([ele, spos, epos]) #removing row of height=1
    whitespace_1.remove([0,0,0])
    l = (len(whitespace_1))
    tex = []
    for i in range(l-1):
        sp = whitespace_1[i][2] #storing start of text block
        ep = whitespace_1[i+1][1] #storing end of text block
        h = hsv[sp:ep]
        tex.append([sp,ep])
        hsv1 = Img.fromarray(h)
        width,height = hsv1.size
        #print(width,height)
        #display(hsv1)
        if height>0 and width>0:
            st1,st2=cols(hsv1,h,height,width,pg_no) #detecting columns and hence text
            f_st1=f_st1+st1 #storing single column and left hand column strings
            f_st2=f_st2+st2 #storing only right hand column strings because no case of single column at end of page has been seen
            #print(st1)
            #print(st2)
    f_st = f_st1+f_st2 #combining text obtained from both columns
    return f_st

In [ ]:
# function to segment columns
def cols(hsv_img,hsv,height,width,pg_no):
    mid=int(width/2)
    element=255
    sp=mid
    ep=mid
    st1 = ""
    st2= ""
    c1=0 #counter to break loop when black pixel has been detected on left side
    c2=0 #same function as c1 but for right side
    for i in range(mid):
        p1 = hsv[:,mid-i] #column left of mid
        p2 = hsv[:,mid+i] #column right of mid
        if c1==0:
            for element in p1: #checking for black pixel on left side
                if element!=255:
                    sp=mid-i
                    c1=1
                    break
        if c2==0:
            for element in p2: #checking for black pixel on left side
                if element!=255:
                    ep=mid+i
                    c2=1
                    break
        if c1==1 and c2==1:#to break outermost loop when columns with black pixel on both sides have been detected
            break
    if (ep-sp)>40:#check for column break
        #print("1")
        hsv1 = hsv[:,0:sp] #croping image for left column
        hsv2 = hsv[:,ep:width] #croping image for right column
        img1 = Img.fromarray(hsv1)
        img2 = Img.fromarray(hsv2)
        st1 = detect_text(img1)#detect text in left cloumn
        st2 = detect_text(img2)#detect text in right column
    else:#if not single column text
        #print("2")
        st1 = detect_text(hsv_img)
    return st1,st2

In [ ]:
#detect text in passed image
def detect_text(hsv_img):
  #print("in text")
  resized_block = hsv_img.resize((hsv_img.width * 4, hsv_img.height * 4))
  text = pytesseract.image_to_string(resized_block, lang='eng',config= '--psm 3 --oem 1')
  return text

In [ ]:
article_dict = {'id':[],
                'text':[],
                'title':[],
                'abstract':[]}

In [ ]:
text_corpora = ""

In [ ]:
def text_extraction():
  global article_dict, text_corpora
  pdf_dir = '/content/drive/MyDrive/UG_ML/Testing'
  pdf_file_no=1
  for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
      print(os.path.join(pdf_dir, filename))
      pdf_file = os.path.join(pdf_dir, filename)
      images = convert_from_path(pdf_file)
      text = ""
      pg_no=0
      for image in images:
        pg_no+=1
        img_array = np.array(image)
        hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        #print(hsv)
        hsv_img = Img.fromarray(hsv)
        #display(hsv_img)
        row = np.size(hsv, axis=0) #calculating rows
        col = np.size(hsv, axis=1) #calculating columns
        tex = rows(hsv,row,hsv_img,pg_no) #pagewise text extraction
        #print(tex)
        text=text+tex #store text as extracted page by page
      text_corpora = text_corpora + text
      article_dict['id'].append(pdf_file_no)
      article_dict['text'].append(text)
      article_dict['title'].append("testing")
      article_dict['abstract'].append("testing")
      pdf_file_no += 1
    # only for testing purpose
    # if(pdf_file_no > 1):
    #   break

In [ ]:
# calling the function
text_extraction()

/content/drive/MyDrive/UG_ML/Testing/Aromatized liposomes for sustained drug.pdf
/content/drive/MyDrive/UG_ML/Testing/Natural and synthetic biomaterials for controlled drug delivery.pdf
/content/drive/MyDrive/UG_ML/Testing/Stimuli‐Responsive Biomaterials  Scaffolds for Stem Cell Control.pdf
/content/drive/MyDrive/UG_ML/Testing/Effect of the nanomicroscale structure of biomaterial scaffolds on bone regeneration.pdf
/content/drive/MyDrive/UG_ML/Testing/Porosity of 3D biomaterial scaffolds and osteogenesis.pdf
/content/drive/MyDrive/UG_ML/Testing/Combining Stem Cells and Biomaterial.pdf


KeyboardInterrupt: ignored

In [ ]:
text_corpora

'\x0cnature communications\n\x0cArticle\n\x0c\x0cAromatized liposomes for sustained drug\n\x0cdelivery\n\x0c\x0cReceived: 1 November 2022 Yang Li’, Tianjiao Ji\', Matthew Torre’, Rachelle Shao ©\', Yueqin Zheng’,\n\x0cDali Wang’, Xiyu Li\', Andong Liu’, Wei Zhang ©", Xiaoran Deng’, Ran Yan’ &\n\nAccepted: 25 September 2023 Daniel S. Kohane®’\n\x0c\x0cPublished online: 20 October 2023\n\x0c\x0c\x0c® Check for updates Insufficient drug loading and leakage of payload remain major challenges in\n\x0c\x0cdesigning liposome-based drug delivery systems. These phenomena can limit\n\x0cduration of effect and cause toxicity. Targeting the rate-limiting step in drug\n\x0crelease from liposomes, we modify (aromatized) them to have aromatic\n\x0cgroups within their lipid bilayers. Aromatized liposomes are designed with\n\x0csynthetic phospholipids with aromatic groups covalently conjugated onto acyl\n\x0cchains. The optimized aromatized liposome increases drug loading and sig-\n\x0cnificantly decre

In [ ]:
article_dict

# Corpus Generation

In [ ]:
words = brown.words()

print(words[:100])
sentences = brown.sents()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]


In [ ]:
def correct_text(input_text):
    words = nltk.word_tokenize(input_text.lower())
    corrected_words = []

    for word in words:
        if word not in brown.words():
            corrected_words.append("[ERR]")
        else:
            corrected_words.append(word)

    corrected_text = " ".join(corrected_words)
    return corrected_text

In [ ]:
def is_number_token(token):
    try:
        float(token)
        return True
    except ValueError:
        return False

In [ ]:
def preprocess(text):
  text = text.replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")

  # Remove string expressions like [1], [1,2,3,4], etc.
  cleaned_text = re.sub(r'\[\d+(,\s*\d+)*\]', '', text)
  #print(cleaned_text)

  # Tokenize the cleaned text into words
  tokens = word_tokenize(cleaned_text)

  # Convert words to lowercase
  lowercase_tokens = [token.lower() for token in tokens]

  # Remove stopwords and punctuation only tokens
  stop_words = set(stopwords.words('english'))
  punctuations = set(string.punctuation)

  filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.lower() not in punctuations and not is_number_token(token) and len(token) > 1]

  # Lemmatize the tokens
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

  return lemmatized_tokens

In [ ]:
final_tokens0 = preprocess(text_corpora)
print(final_tokens0)

['Journal', 'Biomaterials', '``', 'Applications', 'http', '//jba.sagepub.com/', 'Hydrolytic', 'degradation', 'composite', 'poly', 'L-lactide-co-¢-caprolactone', '70/30', '£-tricalcium', 'phosphate', 'Niina', 'Ahola', 'Minna', 'Veiranto', 'Jaana', 'Rich', 'Alexander', 'Efimov', 'Markus', 'Hannula', 'Jukka', 'Seppala', 'Minna', 'Kellomaki', 'Biomater', 'App', 'published', 'online', 'October', 'http', '//jba.sagepub.com/content/early/2012/09/14/0885328212462258', 'recent', 'version', 'article', 'published', 'Oct', 'Published', '6SAGE', 'htto', '//www.sagepublications.com', 'Additional', 'service', 'information', 'Journal', 'Biomaterials', 'Applications', 'found', 'Email', 'Alerts', 'http', '//jba.sagepub.com/cagji/alerts', 'Subscriptions', 'http', '//jba.sagepub.com/subscriptions', 'Reprints', 'http', '/Awww.sagepub.com/journalsReprints.nav', 'Permissions', 'http', '//www.sagepub.com/journalsPermissions.nav', 'OnlineFirst', 'Version', 'Record', 'Oct', 'Article', 'Author', 'Hydrolytic', 'd

In [ ]:
file_path = 'Material_properties.txt'  # Replace with the actual path of your text file
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
final_tokens1 = preprocess(text)
print(final_tokens1)

In [ ]:
file_path = 'Biomaterials_corpus.txt'  # Replace with the actual path of your text file
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
final_tokens2 = preprocess(text)
print(final_tokens2)

In [ ]:
urls = ["https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-a/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-b/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-c/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-d/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-e/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-e/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-f/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-g/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-h/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-i/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-j/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-k/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-l/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-m/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-n/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-o/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-p/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-q/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-r/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-s/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-s/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-s/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-t/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-u/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-v/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-w/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-x/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-x/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-y/","https://www.dierk-raabe.com/glossary-of-materials-science/materials-science-glossary-z/"]

# Variable to store the scraped text
scraped_text = ""

# Iterate over the URLs
for url in urls:
    # Send an HTTP GET request to the webpage
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, "html.parser")
        paragraphs = soup.find_all("p")

        # Extract the text from the HTML elements
        scraped_text += "\n".join([p.get_text() for p in paragraphs]) + "\n"
    else:
        # Print an error message if the request was unsuccessful
        print("Failed to retrieve the webpage. Status code:", response.status_code)

In [ ]:
final_tokens3 = preprocess(scraped_text)
print(final_tokens3)

['Ab-initio', 'lat', '``', 'beginning', "''", 'parameter', 'free', 'method', 'based', 'quantum', 'mechanic', 'electrodynamics', 'solely', 'universal', 'fundamental', 'physical', 'constant', 'needed', 'term', 'ab', 'initio', 'may', 'used', 'different', 'context', 'natural', 'science', 'technical', 'point', 'view', 'following', 'calculation', 'said', '``', 'ab', 'initio', "''", '``', 'first', 'principle', "''", 'relies', 'basic', 'established', 'law', 'nature', 'without', 'additional', 'assumption', 'special', 'model', 'contrast', 'practice', 'according', 'measurement', 'difference', 'ab', 'initio', 'calculation', 'experimental', 'result', 'may', 'show', 'mistake', 'experimental', 'set', 'may', 'deliver', 'accurate', 'determination', 'influencing', 'parameter', 'display', 'far', 'unknown', 'effect', 'use', 'measured', 'value', 'called', 'semi-empirical', 'calculation', 'chemistry', 'two', 'field', 'use', 'term', 'ab', 'initio', 'synthesis', 'mean', 'production', 'chemical', 'compound', '

# Relevancy Score using Bert

In [ ]:
def test_function():
  # Set the folder path containing the PDFs
  folder_path = '/content/drive/MyDrive/UG_ML/Testing'

  # Initialize an empty dataframe
  df = pd.DataFrame(columns=['PDF', 'Abstract'])

  # Loop through each PDF in the folder
  for filename in os.listdir(folder_path):
      if filename.endswith('.pdf'):
          pdf_path = os.path.join(folder_path, filename)

          # Convert the first page of the PDF to an image
          images = convert_from_path(pdf_path, first_page=1, last_page=2)
          image_path = '/content/first_page.jpg'
          images[0].save(image_path, 'JPEG')
          time.sleep(1)

          # Perform OCR on the image
          extracted_text = pytesseract.image_to_string(Img.open(image_path))

          # Find the abstract section on the first page
          start_index = extracted_text.find('Abstract') + len('Abstract')
          end_index = extracted_text.find('Introduction')

          if start_index == -1 or end_index == -1:
              # Abstract and/or Introduction not found on first page
              # Find the abstract section on the second page
              extracted_text = pytesseract.image_to_string(images[1])

              start_index = extracted_text.find('Abstract') + len('Abstract')
              end_index = extracted_text.find('Introduction')

              if start_index == -1 or end_index == -1:
                  # Abstract and/or Introduction not found on second page
                  abstract = 'NaN'
              else:
                  # Extract abstract from the second page
                  abstract = extracted_text[start_index:end_index].strip()
          else:
              # Extract abstract from the first page
              abstract = extracted_text[start_index:end_index].strip()

          # Add the abstract and PDF name to the dataframe
          df = df.append({'PDF': filename, 'Abstract': abstract}, ignore_index=True)
  return df

In [ ]:
data_df = test_function()
#data_df = pd.DataFrame({'PDF': article_dict['title'], 'Abstract': article_dict['abstract']})

<ipython-input-42-c0fa21212c86>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'PDF': filename, 'Abstract': abstract}, ignore_index=True)
<ipython-input-42-c0fa21212c86>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'PDF': filename, 'Abstract': abstract}, ignore_index=True)
<ipython-input-42-c0fa21212c86>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'PDF': filename, 'Abstract': abstract}, ignore_index=True)
<ipython-input-42-c0fa21212c86>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'PDF': filename, 'Abstract': abstract}, ignore_index=True)
<ipython-input-4

In [ ]:
# data_df[data_df['Abstract'] != 'NaN'].head(10)

In [ ]:
# These tags are taken from the corpus generation part
list_of_tags = ['Increasing', 'importance', 'placed', 'hydroxyapatite-derived', 'scaffold', 'bone', 'tissue', 'regeneration', 'application', 'alternative', 'bone', 'graft', 'Insufficient', 'worldwide', 'donor', 'potential', 'risk', 'disease', 'transmission', 'affirm', 'autograft', 'allograft', 'sustainable', 'approach', 'bone', 'substitute', 'Furthermore', 'hydroxyapatite', 'biological', 'similarity', 'bone', 'tissue', 'well', 'abundantly', 'available', 'offering', 'environmentally', 'friendly', 'solution', 'biomedical', 'chemical', 'material', 'ceramic', 'metal', 'polymer', 'bio', 'reaction', 'composite']

In [ ]:
labeled_data = []

In [ ]:
counter = 0
for abstract in data_df['Abstract']:
  print(counter)
  counter = counter + 1
  # if counter < 10:
  #   continue
  # else :
  sentences = sent_tokenize(abstract)
  for sentence in sentences:
    if any(word in sentence for word in list_of_tags):
        label = 1
    else:
        label = 0

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, max_length=31, pad_to_max_length=True)
    labeled_data.append((input_ids, label))

In [ ]:
# data_df

,PDF,Abstract
0,Hydrolytic degradation of composites of poly(L...,There is an increasing need for synthetic bone...


In [ ]:
inputs = torch.tensor([x[0] for x in labeled_data])
labels = torch.tensor([x[1] for x in labeled_data])
dataset = TensorDataset(inputs, labels)
sampler = RandomSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=32)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
for epoch in range(10):
    running_loss = 0.0
    for i, batch in enumerate(dataloader):

        optimizer.zero_grad()

        loss = model(batch[0], labels=batch[1])[0]

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1, Loss: 0.5754251658916474
Epoch 2, Loss: 0.41170135140419006
Epoch 3, Loss: 0.24510425478219985
Epoch 4, Loss: 0.11897285208106041
Epoch 5, Loss: 0.0671571757644415
Epoch 6, Loss: 0.03133349791169167
Epoch 7, Loss: 0.017010646779090165
Epoch 8, Loss: 0.010133611643686891
Epoch 9, Loss: 0.006989490101113915
Epoch 10, Loss: 0.0052782428916543726


In [ ]:
import torch
torch.save(model, '/content/model1.pth')
model.save_pretrained('/content/results/tokenizer/')

# saved_model = torch.load('path/to/model')

In [ ]:
model = torch.load('/content/reseractor_model.pth')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
data_df.head()

,PDF,Abstract
0,Hydrolytic degradation of composites of poly(L...,There is an increasing need for synthetic bone...
1,Current application of β-tricalcium phosphate ...,NaN
2,phase transistion behaviour of TCP.pdf,": In this study, the tricalcium phosphate [B-C..."
3,Preparation of Porous Poly(Lactic Acid)-Trical...,: The development of bioactive and composite m...
4,QUANTITATIVE ANALYSIS OF CRYOMILLED NANOCRYSTA...,"sopisma.pan.pl P,\nse\n\nN www, journals.pan.p..."


In [ ]:
prediction_list = []
prediction_score = []
for abstract in data_df['Abstract']:
  predicted_labels = []

  all_inputs=[]
  t_sum = 0
  f_sum = 0
  sentences = sent_tokenize(abstract)
  for sentence in sentences:

    input_ids = tokenizer.encode(sentence, add_special_tokens=True)


    inputs = torch.tensor(input_ids).unsqueeze(0)


    with torch.no_grad():
        outputs = model(inputs)
        _, predicted_label = torch.max(outputs[0], 1)

    all_inputs.append(inputs)
    predicted_labels.append(predicted_label.item())
    if(predicted_label.item() == 1):
      t_sum += 1
    else:
      f_sum += 1
  score = t_sum/(t_sum + f_sum)
  prediction_score.append(score)
  prediction_list.append(predicted_labels)

In [ ]:
prediction_score

[0.0,
 0.75,
 0.8333333333333334,
 0.0,
 0.8666666666666667,
 0.0,
 1.0,
 1.0,
 0.0,
 0.35294117647058826,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.875,
 0.8333333333333334]

In [ ]:
data_df['p_score'] = prediction_score

In [ ]:
data_df

In [ ]:
vectorizer = Vectorizer()
vectorizer.run(list(data_df['PDF']))
vectors = vectorizer.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


In [ ]:
df4 = pd.DataFrame({'PDF': data_df['PDF'], 'Abstract': data_df['Abstract'], 'title_embedding': vectors})

In [ ]:
df4

,PDF,Abstract,title_embedding
0,Aromatized liposomes for sustained drug.pdf,NaN,"[-0.17888458, -0.36526775, 0.2958917, -0.06887..."
1,Natural and synthetic biomaterials for control...,A wide variety of delivery systems have been\n...,"[-0.20376049, -0.36158448, 0.196011, -0.084536..."
2,Stimuli‐Responsive Biomaterials Scaffolds for...,® Check for updates\n\nADVANCED\nHEALTHCARE\nM...,"[-0.37801534, -0.35306942, 0.21688114, -0.0913..."
3,Effect of the nanomicroscale structure of biom...,NaN,"[-0.44594866, -0.35709584, 0.087655574, -0.185..."
4,Porosity of 3D biomaterial scaffolds and osteo...,Porosity and pore size of biomaterial scaffold...,"[-0.34966582, -0.43098065, 0.19405778, -0.1463..."
5,Combining Stem Cells and Biomaterial.pdf,NaN,"[-0.15442643, -0.4208173, 0.2642932, -0.117621..."
6,Repair of injured spinal cord using biomateria...,The loss of neurons and degeneration of axons ...,"[-0.27428886, -0.34248155, 0.2297592, -0.10246..."
7,Development of biomaterial scaffold for nerve ...,Neural tissue repair and regeneration strategi...,"[-0.36543185, -0.27297476, 0.021785604, -0.131..."
8,Developing a pro-regenerative biomaterial scaf...,NaN,"[-0.41986436, -0.40049422, -0.0016990714, -0.1..."
9,Biomaterials for bone tissue engineering scaff...,OCIETY\nOF CHEMISTRY\n\nRSC Advances\n\nView A...,"[-0.2545388, -0.35157907, 0.14626577, -0.06591..."


In [ ]:
# df4['title_embedding'] = df4['title_embedding'].apply(eval).apply(np.array)

In [ ]:
search_term = input('Enter a search term: ')

Enter a search term: Hydrogel-based scaffolds for tissue engineering


In [ ]:
vectorizer.run(list(search_term))
search_vector = vectorizer.vectors

In [ ]:
search_embeddings = search_vector[-1]

In [ ]:
from numpy.linalg import norm

df4["similarities"] = df4['title_embedding'].apply(lambda x: np.dot(x,search_embeddings)/(norm(x)*norm(search_embeddings)))
data_df['sim_score'] = df4['similarities']

In [ ]:
data_df

,PDF,Abstract,sim_score
0,Aromatized liposomes for sustained drug.pdf,NaN,0.666793
1,Natural and synthetic biomaterials for control...,A wide variety of delivery systems have been\n...,0.698385
2,Stimuli‐Responsive Biomaterials Scaffolds for...,® Check for updates\n\nADVANCED\nHEALTHCARE\nM...,0.717103
3,Effect of the nanomicroscale structure of biom...,NaN,0.782747
4,Porosity of 3D biomaterial scaffolds and osteo...,Porosity and pore size of biomaterial scaffold...,0.751246
5,Combining Stem Cells and Biomaterial.pdf,NaN,0.684091
6,Repair of injured spinal cord using biomateria...,The loss of neurons and degeneration of axons ...,0.713438
7,Development of biomaterial scaffold for nerve ...,Neural tissue repair and regeneration strategi...,0.775916
8,Developing a pro-regenerative biomaterial scaf...,NaN,0.781499
9,Biomaterials for bone tissue engineering scaff...,OCIETY\nOF CHEMISTRY\n\nRSC Advances\n\nView A...,0.721718


In [ ]:
abstract_scores = []
for abstract in df4['Abstract']:
  vectorizer2 = Vectorizer()
  sentences = sent_tokenize(abstract)
  vectorizer.run(sentences)
  vectors = vectorizer.vectors
  abstract_score = [np.dot(x,search_embeddings)/(norm(x)*norm(search_embeddings)) for x in vectors]
  sum = (math.fsum(abstract_score))/len(abstract_score)
  abstract_scores.append(sum)

Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vector

In [ ]:
data_df['abs_score'] = abstract_scores

In [ ]:
ratio_1 = 1
ratio_2 = 4.5
ratio_3 = 4.5

In [ ]:
my_array = [0] * 18
data_df['p_score'] = my_array

In [ ]:
data_df['final_score'] = ratio_1*data_df['p_score'] + ratio_2*data_df['sim_score'] + ratio_3*data_df['abs_score']

In [ ]:
sorted_df = data_df.sort_values(by=['final_score'])
sorted_df.head()

,PDF,Abstract,sim_score,abs_score,p_score,final_score
5,Combining Stem Cells and Biomaterial.pdf,NaN,0.684091,0.862529,0.0,6.959787
14,Programmable nanocomposites of cellulose nanoc...,NaN,0.762162,0.820252,0.0,7.120863
0,Aromatized liposomes for sustained drug.pdf,NaN,0.666793,0.918676,0.0,7.134612
10,Strontium doped bioglass incorporated hydrogel...,NaN,0.792879,0.829264,0.0,7.299644
13,Engineering Biodegradable and Biocompatible Bi...,NaN,0.805814,0.819232,0.0,7.312708


In [ ]:
sorted_df.to_csv("fscores.csv", index = False)

In [ ]:
sorted_df2 = sorted_df[sorted_df['Abstract'] != 'NaN']
sorted_df2.head()

,PDF,Abstract,sim_score,abs_score,p_score,final_score
9,Biomaterials for bone tissue engineering scaff...,OCIETY\nOF CHEMISTRY\n\nRSC Advances\n\nView A...,0.721718,0.828219,0.352941,7.327658
15,synthesis of photo-crosslinkable gelatin metha...,(CTEM ule leased Stale\n\nnucleation and growt...,0.808437,0.820534,0.000000,7.330371
1,Natural and synthetic biomaterials for control...,A wide variety of delivery systems have been\n...,0.698385,0.889555,0.750000,7.895731
16,Control of maleic acid-propylene diepoxide hyd...,al. Biomaterials Research (2022) 26:75\n\nPage...,0.765796,0.814392,0.875000,7.985849
2,Stimuli‐Responsive Biomaterials Scaffolds for...,® Check for updates\n\nADVANCED\nHEALTHCARE\nM...,0.717103,0.876562,0.833333,8.004826


# Extracting testing data from relevant papers

In [ ]:
def taging(lemm):
  tags = nltk.pos_tag(lemm)
  return tags

In [ ]:
#creating sentence tokens
def sen_tok(sents):
  sentences = sent_tokenize(sents)
  r = 0
  final = []
  cd_sent = ""
  ref = ""
  s_tag=[]
  s_pos=[]
  for s in sentences:
    words = word_tokenize(s)
    s_tag = taging(words)
    s_pos = [pos for w, pos in s_tag]
    #separating sentences containing cardinal tags
    if 'CD' in s_pos and r!=1:
      cd_sent+= s
    #separating for references
    if 'References' in words or r==1:
      r=1
      ref+= s
    final+=words
  return ref, final, cd_sent

In [ ]:
#lemmatization
def lemmat(filtered):
  lemma = WordNetLemmatizer()
  lemm = [lemma.lemmatize(word,pos="v") for word in filtered]
  return lemm

In [ ]:
#removing stop words, lemmatizing filtered text
def preprocess(txt):
  stop_words = set(stopwords.words("english"))
  ref, sent_words, cd_sent = sen_tok(txt)
  print(stop_words)
  filtered = [word for word in sent_words if word.casefold() not in stop_words]
  print(filtered)
  lemm = lemmat(filtered)
  return lemm,ref,cd_sent

In [ ]:
#filter lemmatized text from punctuations
def filter(txt):
  #punctuation = ['!','"','#','$','%','&','\'','(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\',']','^','_','`','{','|','}','~']
  import string
  spunct = string.punctuation
  lpunct = []
  lpunct[:0] = spunct
  lpunct+= '±'
  lpunct.remove('%')
  filtered = [tok for tok in txt if tok not in lpunct]
  return filtered

In [ ]:
#take input and create filtered text for analysis
top5_input = ""
for i in range(4,5):
  if(i < len(article_dict['text'])):
    print(i)
    top5_input = top5_input + article_dict['text'][i]
text= []
t_card = ""
references = ""
pgtxt,ref,cd_sent = preprocess(top5_input)
t_card+=cd_sent
references+=ref
text+= pgtxt
f_text = filter(text)
print(pgtxt)
print(len(f_text))

4
{'again', 'doesn', 'needn', 'me', 'over', 'couldn', 'should', 'on', "doesn't", "wouldn't", 'an', "wasn't", 'do', 'don', 'with', 'down', 'same', 'after', 'isn', 'if', 're', 'out', 'where', 'and', 'some', 'is', "needn't", "you've", 'but', 'm', "couldn't", 'it', 'am', 'ours', 'this', 'didn', 'just', 'our', 'whom', 'while', 'against', 'when', 'ourselves', 'off', 'all', 'through', 'does', 'the', 'will', 'she', "she's", 'ma', 'i', 'any', 'shan', 'was', 'own', 'both', 'what', 'yours', 'once', "that'll", 'or', "didn't", 'did', "shan't", 'too', "mustn't", "should've", 'few', 'more', 'be', 'o', 'her', 'wasn', "don't", 'nor', 'than', 'by', 'so', 'for', "aren't", 'other', 'myself', 'their', 'to', "you'll", 'were', 'why', 'itself', 'in', "haven't", 'now', 'up', "shouldn't", 'those', 'have', 'them', 'him', 'before', 'further', 'no', 'y', 'll', 'shouldn', 'wouldn', 'its', 'at', 'd', 'hasn', 'my', 'won', 'such', 'who', 'been', "hadn't", 'most', 'himself', 'he', 'mightn', 'not', 'themselves', 've', '

In [ ]:
#craeting frequency distribution and displaying 30 most frequent words
import matplotlib.pyplot as plt
from nltk import FreqDist
frequency_distribution = FreqDist(f_text)
mc = frequency_distribution.most_common(30)
mc_n = [row[0] for row in mc]
tags = taging(mc_n)
f_mc = [word for word,pos in tags if not ( (len(word)<3) or (pos == 'CD'))]
freq = []
for row in mc:
  if row[0] in f_mc:
    freq.append(row[1])
print(f_mc,"\n",freq)
plt.figure(figsize=(10, 10))
plt.bar(f_mc, freq, color ='blue', width = 0.6)
plt.show()

['scaffold', 'bone', 'cell', 'tissue', 'regeneration', 'use', 'structure', 'cells', 'Mater', 'surface', 'engineer', 'porous', 'differentiation', 'properties', 'pore', 'osteogenic', 'Biomaterials', 'bioactive', 'could', 'biomaterial'] 
 [219, 207, 90, 75, 56, 49, 48, 47, 47, 43, 40, 36, 35, 34, 34, 33, 32, 31, 31, 30]


In [ ]:
#checkboxes for user to choose desired words
from ipywidgets import widgets, Layout
checkboxes = []
for word in f_mc:
  checkbox = widgets.Checkbox(description=word)
  checkboxes.append(checkbox)
widgets.GridBox(checkboxes, layout=Layout(
            width='50%',
            grid_template_rows='auto auto',
            grid_template_columns='25% 30% 25%'))

GridBox(children=(Checkbox(value=False, description='scaffold'), Checkbox(value=False, description='bone'), Ch…

In [ ]:
#extract output obtained from checkboxes
search = []
l = len(checkboxes)
for i in range(0,l):
  if checkboxes[i].value == True:
    search.append(checkboxes[i].description)
print(search)

['scaffold', 'bone', 'tissue', 'regeneration', 'use', 'surface', 'porous', 'properties', 'osteogenic', 'bioactive']


In [ ]:
#extract test conditions based on checkboxes
count = 0
w = -50
l = len(f_text)
units_sear = ['kg','g','kgf','gf','N','kN','mg','°C','μL','ns','g/cm3','kg/m3','K','%','nm','cm','°','h','M','ml','rpm','°/min','mg/ml']
for tok in f_text:
  count+=1
  minimum = max(0,count-50)
  maximum = min(count+50,l)
  if tok in search:
    if count-w<49:
      minimum = w+49
    w=count
    for j in range(minimum,maximum):
      tag = taging([f_text[j]])
      pos = tag[0][1]
      #print(pos)
      if pos == 'CD' and (f_text[j+1]) in units_sear:
        if j>=2 and j<l-2:
          print(tok, ":", f_text[j-2], f_text[j-1], f_text[j], f_text[j+1], f_text[j+2])
        elif j<2:
          print(tok, ":", f_text[j], f_text[j+1], f_text[j+2])
        elif j==l-2:
          print(tok, ":", f_text[j], f_text[j+1])

bone : trabecular structure 75 % -85
bone : porosity composition 87 % HA
regeneration : % HA 13 % 6-TCP
scaffold : % HA 13 % 6-TCP
bone : gl DNA 10 nm ``
bone : f £ 0.1 nm ae
scaffold : 100nm submicronscale 100 nm um
surface : demonstrate presence 10 % nanoparticles
tissue : MPa porosity 83.6 % 0.8
tissue : 83.6 % 0.8 % vitro
scaffold : high porosity 81.98 % 3.75
bioactive : 81.98 % 3.75 % appropriately
scaffold : © Absorbance 570 nm a.u


IndexError: ignored

In [ ]:
!pip install pandas python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=6927714091e412607edba64acf12e7f0406e0c4e045194da0b01b8eadf2da863
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
import pandas as pd
from docx import Document

titles = data_df['PDF']
doc = Document()
for title in titles:
    doc.add_heading(title[:-3], level=1)
count = 0
w = -50
l = len(f_text)
units_sear = ['kg','g','kgf','gf','N','kN','mg','°C','μL','ns','g/cm3','kg/m3','K','%','nm','cm','°','h','M','ml','rpm','°/min','mg/ml']
for tok in f_text:
    count += 1
    minimum = max(0, count - 50)
    maximum = min(count + 50, l)

    if tok in search:
        if count - w < 49:
            minimum = w + 49
        w = count
        for j in range(minimum, maximum):
            tag = taging([f_text[j]])
            pos = tag[0][1]

            if pos == 'CD' and (f_text[j + 1]) in units_sear:
                if j >= 2 and j < l - 2:
                    text = f"{tok} : {f_text[j - 2]} {f_text[j - 1]} {f_text[j]} {f_text[j + 1]} {f_text[j + 2]}"
                elif j < 2:
                    text = f"{tok} : {f_text[j]} {f_text[j + 1]} {f_text[j + 2]}"
                elif j == l - 2:
                    text = f"{tok} : {f_text[j]} {f_text[j + 1]}"

                # Add the text below the header
                doc.add_paragraph(text)

doc.save('extraction.docx')